### <font color='#4285f4'>Overview</font>

Overview: This notebook will deploy the custome ADK agent to agentspace.

Process Flow:
1. Download source code
2. Deploy code to Agent Engine
3. Create Agentspace
4. Associate (deploy) Agent Engine agent to Agentspace 

Cost:
* Approximate cost: $1 (you can use the free agentspace for 30 days)

Author:
* Adam Paternostro

In [ ]:
# Architecture Diagram
from IPython.display import Image
Image(url='https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/deploy-adk-to-agentspace.png', width=1200)

### <font color='#4285f4'>Video Walkthrough</font>

[Video](https://storage.googleapis.com/data-analytics-golden-demo/colab-videos/Demo-Agent-Agentspace.mp4)


In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/data-analytics-golden-demo/colab-videos/Demo-Agent-Agentspace.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

### <font color='#4285f4'>License</font>

```
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

### <font color='#4285f4'>Pip installs - VERY Important (Just run once and restart session)</font>


**PLEASE READ**
These SDKs are changing often and must be installed for the code to work

*   The first block of PIP installs are needed
*   The second block of PIP might not be needed as Colab runtimes are upgraded
*   The third block of PIP might not be needed as Colab runtimes are upgraded

You **MUST** restart the Kernel after this (Runtime | Restart Session)

It might take a few tries for this to work.  



In [ ]:
# Block 1 (needed for Agent)
import sys

!{sys.executable} -m pip install python-dotenv==1.1.1
!{sys.executable} -m pip install json-stream==2.3.3
!{sys.executable} -m pip install tenacity==8.5.0
!{sys.executable} -m pip install google-adk==1.11.0

In [ ]:
# Block 2 (needed to downgrade for Colab)
import sys

!{sys.executable} -m pip install requests==2.32.3 "protobuf<6.0.0"

In [ ]:
# Block 3 (needed to deploy agent using Vertex AI Agent Engine import)
import sys

#!{sys.executable} -m pip uninstall -y vertexai google-cloud-aiplatform
#!{sys.executable} -m pip install --upgrade --user vertexai google-cloud-aiplatform[agent_engines,adk]

!{sys.executable} -m pip install --upgrade vertexai
!{sys.executable} -m pip install --upgrade google-cloud-aiplatform[agent_engines,adk]

### <font color='#4285f4'>Initialize</font>

In [ ]:
from PIL import Image
from IPython.display import HTML
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

import sys
import shutil

import logging
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception

In [ ]:
# Set these (run this cell to verify the output)

agent_engine_region = "us-central1"
project_number = "${project_number}"

agentspace_engine_id = "da_agentspace"
agentspace_display_name = "Data Analytics Agentspace"

code_storage_account = "${data_analytics_agent_code_bucket}"
agent_engine_name = "Data Analytics Agent"


# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = os.environ["GOOGLE_CLOUD_PROJECT"]
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### <font color='#4285f4'>Helper Methods</font>

#### rest_api_helper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def rest_api_helper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import google.auth.transport.requests
  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error rest_api_helper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

### <font color='#4285f4'>Agent Engine - Helper Methods</font>

#### get_agent_engines

In [ ]:
def get_agent_engines() -> dict:
    """
    Lists all the agent engines (Agent Engines).

    Returns:
        dict: A dictionary containing the status and the list of agent engines.
        {
            "status": "success" or "failed",
            "tool_name": "get_agent_engines",
            "query": None,
            "messages": ["List of messages during processing"],
            "results": {
                "reasoningEngines": [ ... list of reasoning engine objects ... ]
            }
        }
    """
    messages = []

    # The URL to list all agent engines in the specified project and region.
    # See the documentation for more details:
    # https://cloud.google.com/vertex-ai/docs/generative-ai/reasoning-engine/list-reasoning-engines
    url = f"https://aiplatform.googleapis.com/v1beta1/projects/{project_id}/locations/{agent_engine_region}/reasoningEngines"

    try:
        json_result = rest_api_helper(url, "GET", None)

        messages.append("Successfully retrieved list of all agent engines from the API.")

        # The API directly returns the list of engines.
        num_engines = len(json_result.get("reasoningEngines", []))
        messages.append(f"Found {num_engines} agent engines.")

        return {
            "status": "success",
            "tool_name": "get_agent_engines",
            "query": None,
            "messages": messages,
            "results": json_result
        }
    except Exception as e:
        messages.append(f"An error occurred while listing agent engines: {e}")
        return {
            "status": "failed",
            "tool_name": "get_agent_engines",
            "query": None,
            "messages": messages,
            "results": None
        }


#### get_agent_engine

In [ ]:
def get_agent_engine(agent_engine_id: str) -> dict:
    """
    Gets a single agent engine (Reasoning Engine) in the configured project and region.

    This function constructs the full resource name from the provided agent_engine_id.
    It returns the full details for the specified engine.

    Args:
        agent_engine_id (str): The name (ID) of the agent engine.
agent_engine_ide.g., "7872226177945960448"

    Returns:
        dict: A dictionary containing the status and the details of the agent engine.
    """
    messages = []

    # The URL to get a specific agent engine in the project and region.
    # Note that agent_engine_name is the final ID in the resource path.
    # See: https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.reasoningEngines/get
    url = f"https://aiplatform.googleapis.com/v1beta1/projects/{project_id}/locations/{agent_engine_region}/reasoningEngines/{agent_engine_id}"

    try:
        # Call the REST API to get the details of the specified agent engine
        json_result = rest_api_helper(url, "GET", None)
        messages.append("Successfully retrieved the agent engine from the API.")

        return {
            "status": "success",
            "tool_name": "get_agent_engine",
            "query": agent_engine_name,
            "messages": messages,
            "results": json_result
        }
    except Exception as e:
        messages.append(f"An error occurred while getting agent engine '{agent_engine_name}': {e}")
        return {
            "status": "failed",
            "tool_name": "get_agent_engine",
            "query": None,
            "messages": messages,
            "results": None
        }

#### exists_agent_engine

In [ ]:
def exists_agent_engine(agent_engine_name: str) -> dict:
    """
    Checks if an agent engine (Agent Engine) already exists by checking the full list.

    This function calls 'get_agent_engines' to retrieve all engines and then checks
    if the specified engine name is in the list.

    Args:
        agent_engine_name (str): The short name/ID of the agent engine.

    Returns:
        dict: A dictionary containing the status and results of the operation.
        {
            "status": "success" or "failed",
            "tool_name": "exists_agent_engine",
            "query": "the-name-of-the-engine-checked",
            "messages": ["List of messages during processing"],
            "results": {
                "exists": True # or False
            }
        }
    """
    # Call the dedicated function to list all agent engines
    list_result = get_agent_engines()
    messages = list_result.get("messages", [])

    # If listing engines failed, propagate the failure
    if list_result["status"] == "failed":
        return {
            "status": "failed",
            "tool_name": "exists_agent_engine",
            "query": agent_engine_name,
            "messages": messages,
            "results": None
        }

    try:
        engine_exists = False
        json_payload = list_result.get("results", {})

        # Construct the full resource name to search for
        full_engine_name_to_find = f"projects/{project_id}/locations/{agent_engine_region}/reasoningEngines/{agent_engine_name}"

        # Loop through the list of engines from the results
        for item in json_payload.get("reasoningEngines", []):
            if item.get("name") == full_engine_name_to_find:
                engine_exists = True
                messages.append(f"Found matching agent engine: '{agent_engine_name}'.")
                break

        if not engine_exists:
            messages.append(f"Agent engine '{agent_engine_name}' does not exist.")

        return {
            "status": "success",
            "tool_name": "exists_agent_engine",
            "query": agent_engine_name,
            "messages": messages,
            "results": {"exists": engine_exists}
        }
    except Exception as e: # Catch potential errors while processing the list
        messages.append(f"An unexpected error occurred while processing the agent engine list: {e}")
        return {
            "status": "failed",
            "tool_name": "exists_agent_engine",
            "query": agent_engine_name,
            "messages": messages,
            "results": None
        }

#### delete_agent_engine

In [ ]:
def delete_agent_engine(agent_engine_id: str, force: bool) -> dict:
    """
    Deletes a specific agent engine (Reasoning Engine).

    This function constructs the full resource name and sends a DELETE request.
    This initiates a long-running operation to delete the engine.

    Args:
        agent_engine_id (str): The unique ID of the agent engine to delete.
                               e.g., "7872226177945960448"
        force: Force the child resources to be deleted

    Returns:
        dict: A dictionary containing the status and the result of the delete operation.
              The result on success is the long-running operation object.
    """
    messages = []

    # The API endpoint for deleting a specific Agent Engine.
    # See: https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.reasoningEngines/delete
    url = f"https://aiplatform.googleapis.com/v1beta1/projects/{project_id}/locations/{agent_engine_region}/reasoningEngines/{agent_engine_id}?force={force}"

    try:
        # Call the REST API to initiate the deletion of the agent engine.
        # This is a long-running operation.
        json_result = rest_api_helper(url, "DELETE", None)
        messages.append(f"Successfully initiated deletion for agent engine '{agent_engine_id}'.")

        return {
            "status": "success",
            "tool_name": "delete_agent_engine",
            "query": agent_engine_id,
            "messages": messages,
            "results": json_result  # This will be the long-running operation object
        }
    except Exception as e:
        messages.append(f"An error occurred while deleting agent engine '{agent_engine_id}': {e}")
        return {
            "status": "failed",
            "tool_name": "delete_agent_engine",
            "query": agent_engine_id,
            "messages": messages,
            "results": None
        }

### <font color='#4285f4'>Agentspace - Helper Methods</font>

#### agentspace_create_placeholder_data_store

In [ ]:
def agentspace_create_placeholder_data_store(
    project_id: str,
    location: str,
    data_store_id: str,
    data_store_display_name: str
) -> dict:
    """
    Creates a placeholder data store in Google Cloud Discovery Engine.

    Args:
        project_id (str): The ID of the Google Cloud project.
        location (str): The location for the data store (e.g., "global").
        data_store_display_name (str): The display name for the new data store.

    Returns:
        dict: A dictionary containing the status and details of the API response.
    """

    # The API endpoint to create a data store.
    url = f"https://discoveryengine.googleapis.com/v1/projects/{project_id}/locations/{location}/collections/default_collection/dataStores?data_store_id={data_store_id}"

    # The request payload for a placeholder data store.
    payload = {
        "displayName": data_store_display_name,
        "industryVertical": "GENERIC",
        "solutionTypes": ["SOLUTION_TYPE_SEARCH"],
        "contentConfig": "NO_CONTENT"
    }


    try:
        # Assuming rest_api_helper handles the authenticated POST request.
        json_result = rest_api_helper(url, "POST", request_body=payload)

        return {
            "status": "success",
            "tool_name": "create_placeholder_data_store",
            "query": payload,
            "results": json_result
        }
    except Exception as e:
        return {
            "status": "failed",
            "tool_name": "create_placeholder_data_store",
            "query": payload,
            "messages": [str(e)],
            "results": None
        }

#### agentspace_create

In [ ]:
def agentspace_create(
    project_id: str,
    location: str,
    agentspace_engine_id: str,
    agentspace_display_name: str,
    placeholder_data_store_id: str
) -> dict:
    """
    Creates an AgentSpace in Google Cloud using a placeholder data store.

    Args:
        project_id (str): The ID of the Google Cloud project.
        location (str): The location for the AgentSpace (e.g., "global").
        agentspace_display_name (str): The display name for the new AgentSpace.
        placeholder_data_store_id (str): The ID of the placeholder data store.

    Returns:
        dict: A dictionary containing the status and details of the API response.
    """
    # The API endpoint to create an engine (AgentSpace).
    url = f"https://discoveryengine.googleapis.com/v1/projects/{project_id}/locations/{location}/collections/default_collection/engines?engineId={agentspace_engine_id}"

    # The request payload, specifying the AgentSpace's properties.
    payload = {
        "displayName": agentspace_display_name,
        "solutionType": "SOLUTION_TYPE_SEARCH",
        "dataStoreIds": [ placeholder_data_store_id ],
        "searchEngineConfig": {
            "searchTier": "SEARCH_TIER_ENTERPRISE",
            "searchAddOns": ["SEARCH_ADD_ON_LLM"]
            },
       "industryVertical": "GENERIC",
       "appType": "APP_TYPE_INTRANET"
    }

    # 'results': {'engines': [{'name': 'projects/517693961302/locations/global/collections/default_collection/engines/adam_agentspace',
    # 'displayName': 'adam_agentspace',
    # 'createTime': '2025-08-29T19:38:57.120442Z',
    # 'dataStoreIds': ['google-drive_1756497235229'],
    # 'solutionType': 'SOLUTION_TYPE_SEARCH',
    # 'searchEngineConfig': {'searchTier': 'SEARCH_TIER_ENTERPRISE',
    # 'searchAddOns': ['SEARCH_ADD_ON_LLM']},
    # 'industryVertical': 'GENERIC',
    # 'appType': 'APP_TYPE_INTRANET'}]}}

    try:
        # Assuming rest_api_helper handles the authenticated POST request.
        json_result = rest_api_helper(url, "POST", request_body=payload)

        return {
            "status": "success",
            "tool_name": "create_agentspace_with_placeholder",
            "query": payload,
            "results": json_result
        }
    except Exception as e:
        return {
            "status": "failed",
            "tool_name": "create_agentspace_with_placeholder",
            "query": payload,
            "messages": [str(e)],
            "results": None
        }

#### agentspace_list

In [ ]:
def agentspace_list(
    project_id: str,
    location: str
) -> dict:
    """
    Lists all engines in a given project and location.

    Args:
        project_id (str): The ID of the Google Cloud project.
        location (str): The location of the engines (e.g., "global").

    Returns:
        dict: A dictionary containing the status and the list of engines.
    """
    # The API endpoint to list engines.
    url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/{location}/collections/default_collection/engines"

    try:
        # Assuming rest_api_helper handles the authenticated GET request.
        json_result = rest_api_helper(url, "GET", None)

        return {
            "status": "success",
            "tool_name": "list_engines",
            "results": json_result
        }
    except Exception as e:
        return {
            "status": "failed",
            "tool_name": "list_engines",
            "messages": [str(e)],
            "results": None
        }

### <font color='#4285f4'>Agentspace to Agent Engine Association - Helper Methods</font>

#### agentspace_associate_agent_engine

In [ ]:
def agentspace_associate_agent_engine(
    project_id: str,
    project_number: str,
    agent_engine_region: str,
    agentspace_engine_id: str,
    agent_engine_id: str,
    agent_display_name: str,
    agent_description: str,
    tool_description: str
) -> dict:
    """
    Associates a provisioned agent engine with an AgentSpace app.

    This function sends a POST request to the Discovery Engine API to create an
    agent within a specified AgentSpace assistant, linking it to a
    pre-existing Agent Engine.

    Args:
        project_id (str): The ID of the Google Cloud project.
            e.g., "data-analytics-agent-test"
        project_number (str): The number of the Google Cloud project.
            e.g., "517693961000"
        agent_engine_region (str): The location of the Agent Engine.
            e.g., "us-central1"
        agentspace_engine_id (str): The ID of the AgentSpace application (engine).
            e.g., "adam_agentspace"
        agent_engine_id (str): The ID of the provisioned Agent Engine.
            e.g., "7872226177945960000"
        agent_display_name (str): The desired display name for the new agent.
            e.g., "Data Analytics Agent"
        agent_description (str): A description for the new agent.
            e.g., "The agent automates data analytics."
        tool_description (str): A description for the agent's tool settings.
            e.g., "Data Analytics Agent for BigQuery, AI Forecast, AI Generate Boolean, Data Profiling, Data Insights"

    Returns:
        dict: A dictionary containing the status and the details of the API response.
    """
    messages = []

    # The API endpoint to create an agent within an assistant in Agentspace.
    url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_number}/locations/global/collections/default_collection/engines/{agentspace_engine_id}/assistants/default_assistant/agents"

    # The request payload, specifying the agent's properties and linking the agent engine.
    payload = {
       "displayName": agent_display_name,
       "description": agent_description,
       "icon": {"uri": "https://storage.googleapis.com/data-analytics-golden-demo/images/ADK-512-color.svg"},
       "adk_agent_definition": {
           "tool_settings": {
               "tool_description": tool_description
           },
           "provisioned_reasoning_engine": {
               "reasoning_engine": f"projects/{project_id}/locations/{agent_engine_region}/reasoningEngines/{agent_engine_id}"
           }
       }
    }

    # Custom headers required by the API. The rest_api_helper is assumed to handle Authorization.
    #headers = {
    #    "x-goog-user-project": project_id,
    #    "Content-Type": "application/json"
    #}

    try:
       # url: str, http_verb: str, request_body: str
        json_result = rest_api_helper(url, "POST", request_body=payload)

        messages.append(f"Successfully sent request to associate agent '{agent_display_name}' with AgentSpace app '{agentspace_engine_id}'.")

        return {
            "status": "success",
            "tool_name": "agentspace_associate_agent_engine",
            "query": payload,
            "messages": messages,
            "results": json_result
        }
    except Exception as e:
        messages.append(f"An error occurred while associating the agent: {e}")
        return {
            "status": "failed",
            "tool_name": "agentspace_associate_agent_engine",
            "query": payload,
            "messages": messages,
            "results": None
        }


#### agentspace_disassociate_agent_engine

In [ ]:
def agentspace_disassociate_agent_engine(
    project_id: str,
    project_number: str,
    agentspace_engine_id: str,
    agent_engine_id: str
) -> dict:
    """
    Disassociates (deletes) an agent from an AgentSpace app.

    This function sends a DELETE request to the Discovery Engine API to remove
    the registration of an agent from a specified AgentSpace assistant.

    Args:
        project_id (str): The ID of the Google Cloud project. This is required
            for the X-Goog-User-Project header.
            e.g., "data-analytics-agent-00000000"
        project_number (str): The number of the Google Cloud project.
            e.g., "517693961000"
        agentspace_engine_id (str): The ID of the AgentSpace application (engine).
            e.g., "adam_agentspace"
        agent_engine_id (str): The ID of the agent to be deleted.
            e.g., "5968342334904680000"

    Returns:
        dict: A dictionary containing the status and the details of the API response.
    """
    messages = []

    # Construct the full resource name for the agent to be deleted.
    agent_resource_name = (
        f"projects/{project_number}/locations/global/collections/"
        f"default_collection/engines/{agentspace_engine_id}/assistants/"
        f"default_assistant/agents/{agent_engine_id}"
    )

    # The API endpoint to delete a specific agent.
    url = f"https://discoveryengine.googleapis.com/v1alpha/{agent_resource_name}"

    try:
        # It is assumed that a 'rest_api_helper' function exists that can handle
        # the REST API call. It should handle authentication and setting the
        # 'x-goog-user-project' header. A DELETE request typically has no body.
        json_result = rest_api_helper(url, "DELETE", None)

        messages.append(f"Successfully sent request to delete agent '{agent_engine_id}' from AgentSpace app '{agentspace_engine_id}'.")

        # A successful DELETE request might return an empty object.
        return {
            "status": "success",
            "tool_name": "agentspace_disassociate_agent_engine",
            "query": None,
            "messages": messages,
            "results": json_result
        }
    except Exception as e:
        messages.append(f"An error occurred while deleting the agent: {e}")
        return {
            "status": "failed",
            "tool_name": "agentspace_disassociate_agent_engine",
            "query": None,
            "messages": messages,
            "results": None
        }

#### agentspace_list_agent_engines

In [ ]:
def agentspace_list_agent_engines(
    project_id: str,
    agentspace_engine_id: str
) -> dict:
    """
    Lists all agents associated with a specific AgentSpace app.

    This function sends a GET request to the Discovery Engine API to retrieve a list
    of all agent registrations within a specified AgentSpace assistant.

    Args:
        project_id (str): The ID of the Google Cloud project. This is required
            for the URL and the X-Goog-User-Project header.
            e.g., "data-analytics-agent-00000000"
        agentspace_engine_id (str): The ID of the AgentSpace application (engine) from which
            to list the agents.
            e.g., "adam_agentspace"

    Returns:
        dict: A dictionary containing the status and the details of the API response,
              with the 'results' key holding a list of agent objects.
    """
    messages = []

    # The API endpoint to list all agents within a specific assistant.
    url = (
        f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/"
        f"collections/default_collection/engines/{agentspace_engine_id}/assistants/"
        f"default_assistant/agents"
    )

    try:
        json_result = rest_api_helper(url, "GET", None)

        messages.append(f"Successfully sent request to list agents from AgentSpace app '{agentspace_engine_id}'.")

        return {
            "status": "success",
            "tool_name": "agentspace_list_agent_engines",
            "query": {"agentspace_engine_id": agentspace_engine_id},
            "messages": messages,
            "results": json_result
        }
    except Exception as e:
        messages.append(f"An error occurred while listing the agents: {e}")
        return {
            "status": "failed",
            "tool_name": "agentspace_list_agent_engines",
            "query": {"agentspace_engine_id": agentspace_engine_id},
            "messages": messages,
            "results": None
        }

### <font color='#4285f4'>MAIN CODE - Deploy Agent Engine and Associate with Agentspace</font>

#### Agent Engine (Examples)

In [ ]:
# To get a list of your Agent Engines
get_agent_engines_response = get_agent_engines()

if "reasoningEngines" not in get_agent_engines_response["results"]:
  print("No Agent Engines")
else:
  for item in get_agent_engines_response["results"]["reasoningEngines"]:
    print(f"Display Nane: {item['displayName']} | Name: {item['name']}")

#print(f"get_agent_engines_response: {json.dumps(get_agent_engines_response, indent=2)}")

In [ ]:
# To get a specific Agent Engine
get_agent_engine_response = get_agent_engine("FILL-ME-IN-WITH-AN-ID")
print(f"get_agent_engine_response: {json.dumps(get_agent_engine_response, indent=2)}")

In [ ]:
# To delete a specific Agent Engine (do not run unless you really want to delete)
# force_delete = True
# delete_agent_engine_response = delete_agent_engine("FILL-ME-IN-WITH-AN-ID",force_delete)
# print(f"delete_agent_engine_response: {json.dumps(delete_agent_engine_response, indent=2)}")

#### Download source code of Data Analytics Agent

In [ ]:
# The souce code is stored in cloud storage as part of the deployment
# Or you can get this from GitHub

gcs_zip_uri = f"gs://{code_storage_account}/data-analytics-agent/data-analytics-agent.zip"
local_zip_file = "data-analytics-agent.zip"

# Use absolute path for a truly isolated workspace
original_directory = os.getcwd()
workspace_dir = os.path.join(original_directory, "agent_deployment_workspace")

# Define all paths relative to the workspace to avoid confusion
local_zip_path = os.path.join(workspace_dir, local_zip_file)
agents_path = os.path.join(workspace_dir, "agents")
source_package_path = os.path.join(agents_path, "data_analytics_agent")

# GCS destination for the final deployment artifacts
gcs_folder = "agent_engine_deployment"
gcs_destination_path = f"gs://{code_storage_account}/{gcs_folder}"

# --- 1. Set up a Clean Workspace ---
print(f"Original directory: {original_directory}")

if os.path.exists(workspace_dir):
    print(f"Removing old workspace: {workspace_dir}")
    shutil.rmtree(workspace_dir)

print(f"Creating new workspace: {workspace_dir}")
os.makedirs(workspace_dir)

# --- 2. Download and Unzip Agent Code into the Workspace ---
print(f"\nDownloading {gcs_zip_uri} to workspace...")
os.system(f"gsutil cp {gcs_zip_uri} {local_zip_path}")

print(f"Unzipping {local_zip_path}...")
# Unzip into the workspace directory. Use the -d flag for destination.
os.system(f"unzip -o {local_zip_path} -d {workspace_dir}")
print("Download and unzip complete.")

In [ ]:
!ls /content/agent_deployment_workspace

In [ ]:
!ls /content/agent_deployment_workspace/agents

In [ ]:
!ls /content/agent_deployment_workspace/agents/data_analytics_agent

#### You must change directories so the Agent deploys

In [ ]:
%cd /content/agent_deployment_workspace/agents

In [ ]:
!pwd

#### Deploy our Agent to Agent Engine

In [ ]:
# https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/deploy

import vertexai
from data_analytics_agent.agent import root_agent
from vertexai import agent_engines

STAGING_BUCKET = f"gs://{code_storage_account}"

# Initialize the Vertex AI SDK
vertexai.init(
    project=project_id,
    location=agent_engine_region,
    staging_bucket=STAGING_BUCKET,
)

# Wrap the agent in an AdkApp object
app = agent_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

# Create Agent Engine
remote_app = agent_engines.create(
    agent_engine=app,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]",
        "json-stream",
        "tenacity",
        "cloudpickle",
        "pydantic"
    ],
    extra_packages=["data_analytics_agent"],
    gcs_dir_name=None,
    display_name=agent_engine_name,
    description="Agent for Google Cloud Data Analytics",
    env_vars={
        "GOOGLE_GENAI_USE_VERTEXAI": "${env_vars_GOOGLE_GENAI_USE_VERTEXAI}",
        # "GOOGLE_CLOUD_PROJECT": "data-analytics-agent-00000000",  # RESERVED ENV (cannot send to agent engine)
        # "GOOGLE_CLOUD_LOCATION": "us-central1",                   # RESERVED ENV (cannot send to agent engine)
        "AGENT_ENV_PROJECT_ID": "${env_vars_AGENT_ENV_PROJECT_ID}",
        "AGENT_ENV_BIGQUERY_REGION": "${env_vars_AGENT_ENV_BIGQUERY_REGION}",
        "AGENT_ENV_DATAPLEX_SEARCH_REGION": "${env_vars_AGENT_ENV_DATAPLEX_SEARCH_REGION}",
        "AGENT_ENV_GOOGLE_API_KEY": "${env_vars_AGENT_ENV_GOOGLE_API_KEY}",
        "AGENT_ENV_DATAPLEX_REGION": "${env_vars_AGENT_ENV_DATAPLEX_REGION}",
        "AGENT_ENV_CONVERSATIONAL_ANALYTICS_REGION": "${env_vars_AGENT_ENV_CONVERSATIONAL_ANALYTICS_REGION}",
        "AGENT_ENV_VERTEX_AI_REGION": "${env_vars_AGENT_ENV_VERTEX_AI_REGION}",
        "AGENT_ENV_BUSINESS_GLOSSARY_REGION": "${env_vars_AGENT_ENV_BUSINESS_GLOSSARY_REGION}",
        "AGENT_ENV_DATAFORM_REGION": "${env_vars_AGENT_ENV_DATAFORM_REGION}",
        "AGENT_ENV_DATAFORM_AUTHOR_NAME": "${env_vars_AGENT_ENV_DATAFORM_AUTHOR_NAME}",
        "AGENT_ENV_DATAFORM_AUTHOR_EMAIL": "${env_vars_AGENT_ENV_DATAFORM_AUTHOR_EMAIL}",
        "AGENT_ENV_DATAFORM_WORKSPACE_DEFAULT_NAME": "${env_vars_AGENT_ENV_DATAFORM_WORKSPACE_DEFAULT_NAME}",
        "AGENT_ENV_DATAFORM_SERVICE_ACCOUNT": "${env_vars_AGENT_ENV_DATAFORM_SERVICE_ACCOUNT}",
        "VERTEX_AI_ENDPOINT": "${env_vars_VERTEX_AI_ENDPOINT}",
        "VERTEX_AI_CONNECTION_NAME": "${env_vars_VERTEX_AI_CONNECTION_NAME}",
        },
    service_account="${env_vars_AGENT_ENGINE_SERVICE_ACCOUNT}"
)

print(f"Deployment finished!")
print(f"Resource Name: {remote_app.resource_name}")

In [ ]:
print(f"To view your Agent Engine: https://console.cloud.google.com/vertex-ai/agents/agent-engines")

#### Test the Agent in Agent Engine

In [ ]:
# Get the deploy agents
get_agent_engines_response = get_agent_engines()

In [ ]:
# Fine the one we just deployed (by name)
agent_engine_id = None
for item in get_agent_engines_response["results"]["reasoningEngines"]:
  print(f"displayName: {item['displayName']}")
  if item["displayName"] == agent_engine_name:
    agent_engine_id = item["name"]

print(f"agent_engine_id: {agent_engine_id}")

In [ ]:
user_id = "user-01"
session_id = "session-01"

In [ ]:
from vertexai import agent_engines
import asyncio

# Create the Agent Engine
remote_app = agent_engines.get(agent_engine_id)

# Create a Session (note: you can see this in the sessions tab in Agent Engine)
remote_session = await remote_app.async_create_session(user_id=user_id)

session_id = remote_session["id"]
print(f"Session object: {remote_session}")
print(f"Session created with ID: {session_id}\n")
print("--- Streaming Query Response ---")

# Now ask a question and see if the Agent replies
async for event in remote_app.async_stream_query(
    user_id=user_id,
    session_id=session_id,
    message="Count the taxi trips by borough",
):
    print(event)

print("\n--- End of Stream ---")

#### Create Agentspace

In [ ]:
agentspace_list(project_id, "global")

In [ ]:
# We need at least one datastore, so just create a placeholder
placeholder_data_store_id = "da_agent_datastore"
placeholder_data_store_display_name = "placeholder"

# NOTE: This might already exists if you re-run this cell, that is fine.
agentspace_create_placeholder_data_store_response = agentspace_create_placeholder_data_store(project_id, "global", placeholder_data_store_id ,placeholder_data_store_display_name)

print(f"agentspace_create_placeholder_data_store_response: {agentspace_create_placeholder_data_store_response}")

In [ ]:
# Create the Agentspace instance
agentspace_create_response = agentspace_create(project_id, "global", agentspace_engine_id, agentspace_display_name, placeholder_data_store_id)

print(f"agentspace_create_response: {agentspace_create_response}")

In [ ]:
print(f"You must configure Google as your Identity Provider")
print(f"Click the Integrations menu on the left hand side and select Use Google Identity")
print(f"https://console.cloud.google.com/gen-app-builder/locations/global/engines/da_agentspace/as-overview/identity?project={project_id}")
print()
print(f"To view your Agentspace: https://console.cloud.google.com/gen-app-builder/engines")

#### Associate with Agentspace

In [ ]:
# This requires allowlisting
# If you get Status: '404', that means you are not allowlisted
agent_engine_id = agent_engine_id.split("/")[-1]  # "7872226177945960000"
agent_display_name = "Data Analytics Agent"
agent_description = "A data analytics agent for Google Data Cloud"
tool_description = "Data Analytics Agent for BigQuery, AI Forecast, AI Generate Boolean, Data Profiling, Data Insights"

# To run this function, a 'rest_api_helper' would need to be defined and implemented.
agentspace_associate_agent_engine_response = agentspace_associate_agent_engine(
    project_id=project_id,
    project_number=project_number,
    agent_engine_region=agent_engine_region,
    agentspace_engine_id=agentspace_engine_id,
    agent_engine_id=agent_engine_id,
    agent_display_name=agent_display_name,
    agent_description=agent_description,
    tool_description=tool_description
)

print(f"agentspace_associate_agent_engine_response: {json.dumps(agentspace_associate_agent_engine_response, indent=2)}")

You can now open Agentspace and click Agents on the left hand side and see the Agent.

In [ ]:
print(f"https://console.cloud.google.com/gen-app-builder/locations/global/engines/{agentspace_engine_id}/as-overview/dashboard?project={project_id}")
print()
print("Open the link, then select 'Copy URL' and paste that to a new tab.")
print()
print("You also activate your 30 day free trial of Agentspace.")
print(f"https://console.cloud.google.com/gen-app-builder/settings/license?project={project_id}")
print()
print("You should now see the agent when you click Agents on the left menu.")

# Architecture Diagram
from IPython.display import Image
Image(url='https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/agentspace_custom_agent.png', width=1000)


#### To View all the Agents in Agentspace

In [ ]:
agentspace_list_agent_engines_response = agentspace_list_agent_engines(
    project_id=project_id,
    agentspace_engine_id=agentspace_engine_id
)

for item in agentspace_list_agent_engines_response["results"]["agents"]:
  print(f"Display Name: {item['displayName']} | Name: {item['name']}")

#print(f"agentspace_list_agent_engines_response: {json.dumps(agentspace_list_agent_engines_response, indent=2)}")

#### To Disassociate an Agent with Agentspace

In [ ]:
##########################################
# This is OPTIONAL
##########################################
agent_id_to_delete = "FILL_ME_IN_WITH_AN_ID (int)"

agentspace_disassociate_agent_engine_response = agentspace_disassociate_agent_engine(
    project_id=project_id,
    project_number=project_number,
    agentspace_engine_id=agentspace_engine_id,
    agent_engine_id=agent_id_to_delete
)

print(f"agentspace_disassociate_agent_engine_response: {json.dumps(agentspace_disassociate_agent_engine_response, indent=2)}")

You can now refresh Agentspace and the agent should be removed (it might take a minute to two).